In [1]:
import torch

In [2]:
# raw_megatron = torch.load('/workspace/ckpt/release/768h-12l-12hd-1mbs-512gbs-1mp-16384tokens-256aligns-1024length-1600ws-100000iter-release/iter_0000500/mp_rank_00/model_optim_rng.pt')
raw_megatron = torch.load('/dataset/ee84df8b/release/ProteinLM/pretrain/ckpt/iter_0000005/mp_rank_00/model_optim_rng.pt')
# raw_esm = torch.load('/root/.cache/torch/hub/checkpoints/esm_msa1b_t12_100M_UR50S.pt')
raw_esm = torch.load('/dataset/ee84df8b/release/ProteinLM/pretrain/data/esm_msa1b_t12_100M_UR50S.pt')

In [3]:
raw_esm = torch.load('/dataset/ee84df8b/release/ProteinLM/pretrain/data/esm_msa1b_t12_100M_UR50S.pt')

In [4]:
megatron = raw_megatron['model']
esm = raw_esm['model']
megatron_lm = megatron['language_model']

megatron_embed = megatron_lm['embedding']
megatron_trans = megatron_lm['transformer']
megatron_lm_head = megatron['lm_head']

print('len(megatron_embed) + len(megatron_trans) + len(megatron_lm_head)', \
    len(megatron_embed) + len(megatron_trans) + len(megatron_lm_head))
esm_embed_keys = ['encoder.sentence_encoder.msa_position_embedding', 'encoder.sentence_encoder.embed_tokens.weight', 'encoder.sentence_encoder.embed_positions.weight']
esm_norm_before = ['encoder.sentence_encoder.emb_layer_norm_before.weight', 'encoder.sentence_encoder.emb_layer_norm_before.bias',]
esm_trans_keys = ['encoder.sentence_encoder.layers.0.column_self_attention.layer.k_proj.weight', 'encoder.sentence_encoder.layers.0.column_self_attention.layer.k_proj.bias', 'encoder.sentence_encoder.layers.0.column_self_attention.layer.v_proj.weight', 'encoder.sentence_encoder.layers.0.column_self_attention.layer.v_proj.bias', 'encoder.sentence_encoder.layers.0.column_self_attention.layer.q_proj.weight', 'encoder.sentence_encoder.layers.0.column_self_attention.layer.q_proj.bias', 'encoder.sentence_encoder.layers.0.column_self_attention.layer.out_proj.weight', 'encoder.sentence_encoder.layers.0.column_self_attention.layer.out_proj.bias', 'encoder.sentence_encoder.layers.0.column_self_attention.layer_norm.weight', 'encoder.sentence_encoder.layers.0.column_self_attention.layer_norm.bias', 'encoder.sentence_encoder.layers.0.row_self_attention.layer.k_proj.weight', 'encoder.sentence_encoder.layers.0.row_self_attention.layer.k_proj.bias', 'encoder.sentence_encoder.layers.0.row_self_attention.layer.v_proj.weight', 'encoder.sentence_encoder.layers.0.row_self_attention.layer.v_proj.bias', 'encoder.sentence_encoder.layers.0.row_self_attention.layer.q_proj.weight', 'encoder.sentence_encoder.layers.0.row_self_attention.layer.q_proj.bias', 'encoder.sentence_encoder.layers.0.row_self_attention.layer.out_proj.weight', 'encoder.sentence_encoder.layers.0.row_self_attention.layer.out_proj.bias', 'encoder.sentence_encoder.layers.0.row_self_attention.layer_norm.weight', 'encoder.sentence_encoder.layers.0.row_self_attention.layer_norm.bias', 'encoder.sentence_encoder.layers.0.feed_forward_layer.layer.fc1.weight', 'encoder.sentence_encoder.layers.0.feed_forward_layer.layer.fc1.bias', 'encoder.sentence_encoder.layers.0.feed_forward_layer.layer.fc2.weight', 'encoder.sentence_encoder.layers.0.feed_forward_layer.layer.fc2.bias', 'encoder.sentence_encoder.layers.0.feed_forward_layer.layer_norm.weight', 'encoder.sentence_encoder.layers.0.feed_forward_layer.layer_norm.bias']
esm_lm_head_keys = ['encoder.lm_head.weight', 'encoder.lm_head.bias', 'encoder.lm_head.dense.weight', 'encoder.lm_head.dense.bias', 'encoder.lm_head.layer_norm.weight', 'encoder.lm_head.layer_norm.bias']
esm_norm_after = ['encoder.sentence_encoder.emb_layer_norm_after.weight', 'encoder.sentence_encoder.emb_layer_norm_after.bias', ]

print('len(esm_embed_keys) + 12 * len(esm_trans_keys) + len(esm_lm_head_keys)', \
    len(esm_embed_keys) + len(esm_norm_before) + 12 * len(esm_trans_keys) + len(esm_norm_after) + len(esm_lm_head_keys))

mega_embed_keys = [i for i in megatron_embed.keys()]

mega_trans_keys = [i for i in megatron_trans.keys() if 'layers.0' in i]
mega_lm_head_keys = [i for i in megatron_lm_head.keys()] + ['final_layernorm.weight', 'final_layernorm.bias']

len(megatron_embed) + len(megatron_trans) + len(megatron_lm_head) 227
len(esm_embed_keys) + 12 * len(esm_trans_keys) + len(esm_lm_head_keys) 325


In [5]:
megatron['language_model']['embedding'].keys()

dict_keys(['word_embeddings', 'position_embeddings', 'msa_positional_embeddings', 'emb_layer_norm_before'])

In [6]:
megatron['language_model']['embedding']['emb_layer_norm_before'].keys()

odict_keys(['weight', 'bias'])

In [7]:
for i in range(3):
    print(esm_embed_keys[i], esm[esm_embed_keys[i]].shape)

for i in range(len(mega_embed_keys)):
    print(mega_embed_keys[i], megatron_embed[mega_embed_keys[i]]['weight'].shape)

encoder.sentence_encoder.msa_position_embedding torch.Size([1, 1024, 1, 768])
encoder.sentence_encoder.embed_tokens.weight torch.Size([33, 768])
encoder.sentence_encoder.embed_positions.weight torch.Size([1026, 768])
word_embeddings torch.Size([128, 768])
position_embeddings torch.Size([1026, 768])
msa_positional_embeddings torch.Size([1024, 768])
emb_layer_norm_before torch.Size([768])


In [8]:
def assign(dst, src):
    assert src.shape == dst.shape
    dst[:] = src.to(dst.device)[:]
print(mega_embed_keys, esm_embed_keys + esm_norm_before)

['word_embeddings', 'position_embeddings', 'msa_positional_embeddings', 'emb_layer_norm_before'] ['encoder.sentence_encoder.msa_position_embedding', 'encoder.sentence_encoder.embed_tokens.weight', 'encoder.sentence_encoder.embed_positions.weight', 'encoder.sentence_encoder.emb_layer_norm_before.weight', 'encoder.sentence_encoder.emb_layer_norm_before.bias']


In [9]:
def assign_embed():
    m_id_e_id = {2: 0, 0: 1, 1: 32, 5: 5, 6: 25, 7: 23, 8: 13, 9: 9, 10: 18, 11: 6, 12: 21, 13: 12, 14: 15, 15: 4, 16: 20, 17: 17, 18: 28, 19: 14, 20: 16, 21: 10, 22: 8, 23: 11, 24: 26, 25: 7, 26: 22, 27: 24, 28: 19, 29: 27, 30: 30}
    for m, e in m_id_e_id.items():
        assign(megatron_embed['word_embeddings']['weight'][m], esm['encoder.sentence_encoder.embed_tokens.weight'][e])
    assign(megatron_embed['word_embeddings']['weight'][: 33], esm['encoder.sentence_encoder.embed_tokens.weight'])
    assign(megatron_embed['position_embeddings']['weight'], esm['encoder.sentence_encoder.embed_positions.weight'])
    assign(megatron_embed['msa_positional_embeddings']['weight'], esm['encoder.sentence_encoder.msa_position_embedding'][0, :, 0])

    assign(megatron_embed['emb_layer_norm_before']['weight'], esm['encoder.sentence_encoder.emb_layer_norm_before.weight'])
    assign(megatron_embed['emb_layer_norm_before']['bias'], esm['encoder.sentence_encoder.emb_layer_norm_before.bias'])

assign_embed()

In [10]:

megatron_embed['word_embeddings']['weight'][0].sum()

tensor(5.7588, device='cuda:0')

In [11]:
# torch.save(raw_megatron, '/dataset/ee84df8b/release/ProteinLM/pretrain/dump/iter_0000010/mp_rank_00/model_optim_rng.pt')

In [12]:
dump = torch.load('/dataset/ee84df8b/release/ProteinLM/pretrain/dump/iter_0000010/mp_rank_00/model_optim_rng.pt')
dump['model']['language_model']['embedding']['word_embeddings']['weight'][:2], \
    raw_megatron['model']['language_model']['embedding']['word_embeddings']['weight'][:2]

(tensor([[-0.2673, -0.0990,  0.1875,  ..., -0.1162,  0.0708, -0.2198],
         [-0.0086, -0.0358,  0.0795,  ..., -0.3044,  0.0036, -0.0347]],
        device='cuda:0'),
 tensor([[-0.2673, -0.0990,  0.1875,  ..., -0.1162,  0.0708, -0.2198],
         [-0.0086, -0.0358,  0.0795,  ..., -0.3044,  0.0036, -0.0347]],
        device='cuda:0'))

In [13]:
raw_megatron['model']['language_model']['embedding']['word_embeddings']['weight'][:33].sum(), \
    raw_megatron['model']['language_model']['embedding']['position_embeddings']['weight'].sum()

(tensor(nan, device='cuda:0'), tensor(47.9429, device='cuda:0'))

In [14]:
esm['encoder.sentence_encoder.embed_tokens.weight'].sum(), esm['encoder.sentence_encoder.embed_positions.weight'].sum()

(tensor(-34.7428), tensor(47.9429))

## process layernorm before

# Process Tansformer

In [15]:
megatron_trans['layers.0.row_input_layernorm.weight'].shape, \
    esm['encoder.sentence_encoder.layers.0.column_self_attention.layer_norm.weight'].shape

(torch.Size([768]), torch.Size([768]))

In [16]:
megatron_trans['layers.0.row_attention.query_key_value.weight'].shape, \
    esm['encoder.sentence_encoder.layers.0.column_self_attention.layer.k_proj.weight'].shape

(torch.Size([2304, 768]), torch.Size([768, 768]))

In [17]:
mega_trans_keys

['layers.0.row_input_layernorm.weight',
 'layers.0.row_input_layernorm.bias',
 'layers.0.col_input_layernorm.weight',
 'layers.0.col_input_layernorm.bias',
 'layers.0.row_attention.query_key_value.weight',
 'layers.0.row_attention.query_key_value.bias',
 'layers.0.row_attention.dense.weight',
 'layers.0.row_attention.dense.bias',
 'layers.0.col_attention.query_key_value.weight',
 'layers.0.col_attention.query_key_value.bias',
 'layers.0.col_attention.dense.weight',
 'layers.0.col_attention.dense.bias',
 'layers.0.post_attention_layernorm.weight',
 'layers.0.post_attention_layernorm.bias',
 'layers.0.mlp.dense_h_to_4h.weight',
 'layers.0.mlp.dense_h_to_4h.bias',
 'layers.0.mlp.dense_4h_to_h.weight',
 'layers.0.mlp.dense_4h_to_h.bias']

In [18]:
esm_trans_keys

['encoder.sentence_encoder.layers.0.column_self_attention.layer.k_proj.weight',
 'encoder.sentence_encoder.layers.0.column_self_attention.layer.k_proj.bias',
 'encoder.sentence_encoder.layers.0.column_self_attention.layer.v_proj.weight',
 'encoder.sentence_encoder.layers.0.column_self_attention.layer.v_proj.bias',
 'encoder.sentence_encoder.layers.0.column_self_attention.layer.q_proj.weight',
 'encoder.sentence_encoder.layers.0.column_self_attention.layer.q_proj.bias',
 'encoder.sentence_encoder.layers.0.column_self_attention.layer.out_proj.weight',
 'encoder.sentence_encoder.layers.0.column_self_attention.layer.out_proj.bias',
 'encoder.sentence_encoder.layers.0.column_self_attention.layer_norm.weight',
 'encoder.sentence_encoder.layers.0.column_self_attention.layer_norm.bias',
 'encoder.sentence_encoder.layers.0.row_self_attention.layer.k_proj.weight',
 'encoder.sentence_encoder.layers.0.row_self_attention.layer.k_proj.bias',
 'encoder.sentence_encoder.layers.0.row_self_attention.lay

In [19]:
def process_layer_i(i: int):
    for p in ['weight', 'bias']:
        for rc in ['row', 'col']:
            esm_rc = 'column' if rc == 'row' else 'row'
            assign(megatron_trans[f'layers.{i}.{rc}_input_layernorm.{p}'], esm[f'encoder.sentence_encoder.layers.{i}.{esm_rc}_self_attention.layer_norm.{p}'])
            assign(megatron_trans[f'layers.{i}.{rc}_attention.dense.{p}'], esm[f'encoder.sentence_encoder.layers.{i}.{esm_rc}_self_attention.layer.out_proj.{p}'])

        assign(megatron_trans[f'layers.{i}.mlp.dense_h_to_4h.{p}'], esm[f'encoder.sentence_encoder.layers.{i}.feed_forward_layer.layer.fc1.{p}'])
        assign(megatron_trans[f'layers.{i}.mlp.dense_4h_to_h.{p}'], esm[f'encoder.sentence_encoder.layers.{i}.feed_forward_layer.layer.fc2.{p}'])
        assign(megatron_trans[f'layers.{i}.post_attention_layernorm.{p}'], esm[f'encoder.sentence_encoder.layers.{i}.feed_forward_layer.layer_norm.{p}'])

    num_heads = 12
    hidden_dim = 768
    heads_dim = hidden_dim // num_heads
    for rc in ['row', 'col']:
        # esm_rc = rc if rc == 'row' else 'column'
        esm_rc = 'column' if rc == 'row' else 'row'
        # .contiguous()
        wq = esm[f'encoder.sentence_encoder.layers.{i}.{esm_rc}_self_attention.layer.q_proj.weight'].view(num_heads, heads_dim, -1)
        wk = esm[f'encoder.sentence_encoder.layers.{i}.{esm_rc}_self_attention.layer.k_proj.weight'].view(num_heads, heads_dim, -1)
        wv = esm[f'encoder.sentence_encoder.layers.{i}.{esm_rc}_self_attention.layer.v_proj.weight'].view(num_heads, heads_dim, -1)
        bq = esm[f'encoder.sentence_encoder.layers.{i}.{esm_rc}_self_attention.layer.q_proj.bias'].view(num_heads, heads_dim)
        bk = esm[f'encoder.sentence_encoder.layers.{i}.{esm_rc}_self_attention.layer.k_proj.bias'].view(num_heads, heads_dim)
        bv = esm[f'encoder.sentence_encoder.layers.{i}.{esm_rc}_self_attention.layer.v_proj.bias'].view(num_heads, heads_dim)
        # print(wq.shape, bq.shape)
        # torch.Size([12, 64, 768]) torch.Size([12, 64])
        W_mixed = torch.cat((wq, wk, wv), dim=1).reshape(hidden_dim * 3, hidden_dim)
        B_mixed = torch.cat((bq, bk, bv), dim=1).reshape(-1)
        assign(megatron_trans[f'layers.{i}.{rc}_attention.query_key_value.weight'], W_mixed)
        assign(megatron_trans[f'layers.{i}.{rc}_attention.query_key_value.bias'], B_mixed)
for i in range(12):
    process_layer_i(i)
torch.save(raw_megatron, '/dataset/ee84df8b/release/ProteinLM/pretrain/dump/iter_0000010/mp_rank_00/model_optim_rng.pt')


In [33]:
process_layer_i(11)

In [55]:
esm['encoder.sentence_encoder.embed_tokens.weight'].shape

torch.Size([33, 768])

In [31]:
# megatron_trans.keys()

In [62]:
fs = "UniRef50-xa-a2m-2017/ UniRef50-xb-a2m-2018/ UniRef50-xc-a2m-2017/ UniRef50-xd-a2m-2018/ UniRef50-xe-a2m-2017/ UniRef50-xf-a2m-2018"
folders = [i.strip() for i in fs.split('/')]
folders = ['/workspace/data/' + f + '/' + f + '.json' for f in folders]

print(f'/bin/cat {" ".join(folders)} > /workspace/data/TOTAL.jsonl')

/bin/cat /workspace/data/UniRef50-xa-a2m-2017/UniRef50-xa-a2m-2017.json /workspace/data/UniRef50-xb-a2m-2018/UniRef50-xb-a2m-2018.json /workspace/data/UniRef50-xc-a2m-2017/UniRef50-xc-a2m-2017.json /workspace/data/UniRef50-xd-a2m-2018/UniRef50-xd-a2m-2018.json /workspace/data/UniRef50-xe-a2m-2017/UniRef50-xe-a2m-2017.json /workspace/data/UniRef50-xf-a2m-2018/UniRef50-xf-a2m-2018.json > /workspace/data/TOTAL.jsonl


In [73]:
isinstance(esm['encoder.sentence_encoder.layers.0.feed_forward_layer.layer_norm.bias'], torch.Tensor)

True

In [83]:
tot = 0
def recursive_print_param_shape(model_dict):
    for k in model_dict:
        if isinstance(model_dict[k], torch.Tensor):
            print(k, model_dict[k].shape)
            global tot
            tot += model_dict[k].numel()
        else:
            recursive_print_param_shape(model_dict[k])
    # print(tot)

In [81]:
recursive_print_param_shape(esm)

encoder.sentence_encoder.msa_position_embedding torch.Size([1, 1024, 1, 768])
encoder.sentence_encoder.embed_tokens.weight torch.Size([33, 768])
encoder.sentence_encoder.embed_positions.weight torch.Size([1026, 768])
encoder.sentence_encoder.layers.0.column_self_attention.layer.k_proj.weight torch.Size([768, 768])
encoder.sentence_encoder.layers.0.column_self_attention.layer.k_proj.bias torch.Size([768])
encoder.sentence_encoder.layers.0.column_self_attention.layer.v_proj.weight torch.Size([768, 768])
encoder.sentence_encoder.layers.0.column_self_attention.layer.v_proj.bias torch.Size([768])
encoder.sentence_encoder.layers.0.column_self_attention.layer.q_proj.weight torch.Size([768, 768])
encoder.sentence_encoder.layers.0.column_self_attention.layer.q_proj.bias torch.Size([768])
encoder.sentence_encoder.layers.0.column_self_attention.layer.out_proj.weight torch.Size([768, 768])
encoder.sentence_encoder.layers.0.column_self_attention.layer.out_proj.bias torch.Size([768])
encoder.sentenc

In [82]:
tot

115641633

In [84]:
recursive_print_param_shape(megatron)

weight torch.Size([128, 768])
weight torch.Size([1024, 768])
layers.0.row_input_layernorm.weight torch.Size([768])
layers.0.row_input_layernorm.bias torch.Size([768])
layers.0.col_input_layernorm.weight torch.Size([768])
layers.0.col_input_layernorm.bias torch.Size([768])
layers.0.row_attention.query_key_value.weight torch.Size([2304, 768])
layers.0.row_attention.query_key_value.bias torch.Size([2304])
layers.0.row_attention.dense.weight torch.Size([768, 768])
layers.0.row_attention.dense.bias torch.Size([768])
layers.0.col_attention.query_key_value.weight torch.Size([2304, 768])
layers.0.col_attention.query_key_value.bias torch.Size([2304])
layers.0.col_attention.dense.weight torch.Size([768, 768])
layers.0.col_attention.dense.bias torch.Size([768])
layers.0.post_attention_layernorm.weight torch.Size([768])
layers.0.post_attention_layernorm.bias torch.Size([768])
layers.0.mlp.dense_h_to_4h.weight torch.Size([3072, 768])
layers.0.mlp.dense_h_to_4h.bias torch.Size([3072])
layers.0.mlp.d

In [57]:
tot

NameError: name 'tot' is not defined

In [56]:
!factor 741793 # 115641633 - 114899840

741793: 13 43 1327
